In [12]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
from tensorflow.keras import layers
from keras.callbacks import TensorBoard

In [13]:
# tfds.list_builders()

In [14]:
(ds_train, ds_test), ds_info = tfds.load(
    'kddcup99',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=False,
    with_info=True,
)

Preprocess

In [15]:
ds_info

tfds.core.DatasetInfo(
    name='kddcup99',
    full_name='kddcup99/1.0.1',
    description="""
    This is the data set used for The Third International Knowledge Discovery and
    Data Mining Tools Competition, which was held in conjunction with KDD-99 The
    Fifth International Conference on Knowledge Discovery and Data Mining. The
    competition task was to build a network intrusion detector, a predictive model
    capable of distinguishing between 'bad' connections, called intrusions or
    attacks, and 'good' normal connections. This database contains a standard set of
    data to be audited, which includes a wide variety of intrusions simulated in a
    military network environment.
    """,
    homepage='https://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html',
    data_path='C:\\Users\\EhsanQA\\tensorflow_datasets\\kddcup99\\1.0.1',
    file_format=tfrecord,
    download_size=18.62 MiB,
    dataset_size=5.25 GiB,
    features=FeaturesDict({
        'count': int32,
        '

In [16]:
mins = {'count': 10000,
        'diff_srv_rate': 10000,
        'dst_bytes': 10000,
        'dst_host_count': 10000,
        'dst_host_diff_srv_rate': 10000,
        'dst_host_rerror_rate': 10000,
        'dst_host_same_src_port_rate': 10000,
        'dst_host_same_srv_rate': 10000,
        'dst_host_serror_rate': 10000,
        'dst_host_srv_count': 10000,
        'dst_host_srv_diff_host_rate': 10000,
        'dst_host_srv_rerror_rate': 10000,
        'dst_host_srv_serror_rate': 10000,
        'duration': 10000,
        'hot': 10000,
        'is_guest_login': 10000,
        'is_hot_login': 10000,
        'land': 10000,
        'logged_in': 10000,
        'num_access_files': 10000,
        'num_compromised': 10000,
        'num_failed_logins': 10000,
        'num_file_creations': 10000,
        'num_outbound_cmds': 10000,
        'num_root': 10000,
        'num_shells': 10000,
        'rerror_rate': 10000,
        'root_shell': 10000,
        'same_srv_rate': 10000,
        'serror_rate': 10000,
        'src_bytes': 10000,
        'srv_count': 10000,
        'srv_diff_host_rate': 10000,
        'srv_rerror_rate': 10000,
        'srv_serror_rate': 10000,
        'su_attempted': 10000,
        'urgent': 10000,
        'wrong_fragment': 10000}
maxs = {'count': -10000,
        'diff_srv_rate': -10000,
        'dst_bytes': -10000,
        'dst_host_count': -10000,
        'dst_host_diff_srv_rate': -10000,
        'dst_host_rerror_rate': -10000,
        'dst_host_same_src_port_rate': -10000,
        'dst_host_same_srv_rate': -10000,
        'dst_host_serror_rate': -10000,
        'dst_host_srv_count': -10000,
        'dst_host_srv_diff_host_rate': -10000,
        'dst_host_srv_rerror_rate': -10000,
        'dst_host_srv_serror_rate': -10000,
        'duration': -10000,
        'hot': -10000,
        'is_guest_login': -10000,
        'is_hot_login': -10000,
        'land': -10000,
        'logged_in': -10000,
        'num_access_files': -10000,
        'num_compromised': -10000,
        'num_failed_logins': -10000,
        'num_file_creations': -10000,
        'num_outbound_cmds': -10000,
        'num_root': -10000,
        'num_shells': -10000,
        'rerror_rate': -10000,
        'root_shell': -10000,
        'same_srv_rate': -10000,
        'serror_rate': -10000,
        'src_bytes': -10000,
        'srv_count': -10000,
        'srv_diff_host_rate': -10000,
        'srv_rerror_rate': -10000,
        'srv_serror_rate': -10000,
        'su_attempted': -10000,
        'urgent': -10000,
        'wrong_fragment': -10000}
nos = ['flag', 'services', 'label', 'protocol_type', 'service']        

In [17]:
# for datapoint in ds_train.enumerate():
#     for item in datapoint[1].items():
#         if item[0] not in nos:
#             val = item[1]
#             mins[item[0]] = min(float(val), float(mins[item[0]]))
#             maxs[item[0]] = max(float(val), float(maxs[item[0]]))
# print("THE MINS")
# print(mins)
# print("THE MAXES")
# print(maxs)

In [18]:
mins = {'count': 0.0, 'diff_srv_rate': 0.0, 'dst_bytes': 0.0, 'dst_host_count': 0.0, 'dst_host_diff_srv_rate': 0.0, 'dst_host_rerror_rate': 0.0, 'dst_host_same_src_port_rate': 0.0, 'dst_host_same_srv_rate': 0.0, 'dst_host_serror_rate': 0.0, 'dst_host_srv_count': 0.0, 'dst_host_srv_diff_host_rate': 0.0, 'dst_host_srv_rerror_rate': 0.0, 'dst_host_srv_serror_rate': 0.0, 'duration': 0.0, 'hot': 0.0, 'is_guest_login': 0.0, 'is_hot_login': 0.0, 'land': 0.0, 'logged_in': 0.0, 'num_access_files': 0.0, 'num_compromised': 0.0, 'num_failed_logins': 0.0, 'num_file_creations': 0.0, 'num_outbound_cmds': 0.0, 'num_root': 0.0, 'num_shells': 0.0, 'rerror_rate': 0.0, 'root_shell': 0.0, 'same_srv_rate': 0.0, 'serror_rate': 0.0, 'src_bytes': 0.0, 'srv_count': 0.0, 'srv_diff_host_rate': 0.0, 'srv_rerror_rate': 0.0, 'srv_serror_rate': 0.0, 'su_attempted': 0.0, 'urgent': 0.0, 'wrong_fragment': 0.0}
maxs = {'count': 511.0, 'diff_srv_rate': 1.0, 'dst_bytes': 1309937401.0, 'dst_host_count': 255.0, 'dst_host_diff_srv_rate': 1.0, 'dst_host_rerror_rate': 1.0, 'dst_host_same_src_port_rate': 1.0, 'dst_host_same_srv_rate': 1.0, 'dst_host_serror_rate': 1.0, 'dst_host_srv_count': 255.0, 'dst_host_srv_diff_host_rate': 1.0, 'dst_host_srv_rerror_rate': 1.0, 'dst_host_srv_serror_rate': 1.0, 'duration': 58329.0, 'hot': 77.0, 'is_guest_login': 1.0, 'is_hot_login': 1.0, 'land': 1.0, 'logged_in': 1.0, 'num_access_files': 9.0, 'num_compromised': 7479.0, 'num_failed_logins': 5.0, 'num_file_creations': 43.0, 'num_outbound_cmds': 0.0, 'num_root': 7468.0, 'num_shells': 2.0, 'rerror_rate': 1.0, 'root_shell': 1.0, 'same_srv_rate': 1.0, 'serror_rate': 1.0, 'src_bytes': 1379963888.0, 'srv_count': 511.0, 'srv_diff_host_rate': 1.0, 'srv_rerror_rate': 1.0, 'srv_serror_rate': 1.0, 'su_attempted': 2.0, 'urgent': 14.0, 'wrong_fragment': 3.0}

In [29]:
a = np.zeros((5, 2))
a = np.append(a[0], 9)


array([[1., 1.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.]])

In [20]:
# def preprocess(ds):
dataset = [[0] * 79] * len(ds_train)
# dataset = np.zeros((79, 10000))
print(np.array(dataset).shape)
i = -1
for datapoint in ds_train.enumerate():
    i += 1
    if (i >= 100):
        break
    for item in datapoint[1].items():
        if (item[0] == 'protocol_type'):
            dataset[i][int(item[1])] = 1
        elif (item[0] == 'flag'):
            dataset[i][item[1] + 2] = 1
        elif (item[0] == 'services'):
            dataset[i][item[1] + 14] = 1
        elif (item[0] == 'label'):
            dataset[i].append(-1 * int(item[1]))
        elif item[0] != 'service':
            if maxs[item[0]] - mins[item[0]] != 0:
                dataset[i].append((float(item[1]) - mins[item[0]]) / (maxs[item[0]] - mins[item[0]]))
            else:
                dataset[i].append(float(item[1]))
print(np.array(dataset).shape)
print(dataset[0])

(79, 10000)


AttributeError: 'numpy.ndarray' object has no attribute 'append'

In [ ]:
dataset = np.array(dataset)
np.savetxt("dataset.txt", dataset)
content = np.loadtxt('dataset.txt')

Model

In [6]:
input_layer = keras.Input((117,))
H1 = layers.Dense(32, activation='sigmoid')
# D1 = layers.Dense(32)
H2 = layers.Dense(32, activation='sigmoid')
# D2 = layers.Dense(32)
H3 = layers.Dense(32, activation='sigmoid')
# D3 = layers.Dense(32)
H4 = layers.Dense(32, activation='sigmoid')
# D4 = layers.Dense(32)
supervised_layer = layers.Dense(5, activation='sigmoid')
output_layer = layers.Dense(5, activation='softmax')

AutoEncoder 1

In [8]:
encoded1 = H1(input_layer)
decoded1 = layers.Dense(117,)(encoded1)
auto_encoder1 = keras.Model(input_layer, decoded1)
auto_encoder1.compile()
auto_encoder1.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 117)]             0         
                                                                 
 dense (Dense)               (None, 32)                3776      
                                                                 
 dense_6 (Dense)             (None, 117)               3861      
                                                                 
Total params: 7,637
Trainable params: 7,637
Non-trainable params: 0
_________________________________________________________________


AutoEncoder 2

In [9]:
encoded2 = H2(encoded1)
decoded2 = layers.Dense(32,)(encoded2)
auto_encoder2 = keras.Model(encoded1, decoded2)
auto_encoder2.compile()
auto_encoder2.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 32)]              0         
                                                                 
 dense_1 (Dense)             (None, 32)                1056      
                                                                 
 dense_7 (Dense)             (None, 32)                1056      
                                                                 
Total params: 2,112
Trainable params: 2,112
Non-trainable params: 0
_________________________________________________________________


AutoEncoder 3

In [10]:
encoded3 = H3(encoded2)
decoded3 = layers.Dense(32,)(encoded3)
auto_encoder3 = keras.Model(encoded2, decoded3)
auto_encoder3.compile()
auto_encoder3.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 32)]              0         
                                                                 
 dense_2 (Dense)             (None, 32)                1056      
                                                                 
 dense_8 (Dense)             (None, 32)                1056      
                                                                 
Total params: 2,112
Trainable params: 2,112
Non-trainable params: 0
_________________________________________________________________


AutoEncoder 4

In [11]:
encoded4 = H4(encoded3)
decoded4 = layers.Dense(32,)(encoded4)
auto_encoder4 = keras.Model(encoded3, decoded4)
auto_encoder4.compile()
auto_encoder4.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 32)]              0         
                                                                 
 dense_3 (Dense)             (None, 32)                1056      
                                                                 
 dense_9 (Dense)             (None, 32)                1056      
                                                                 
Total params: 2,112
Trainable params: 2,112
Non-trainable params: 0
_________________________________________________________________


Train

In [12]:
ds_info

tfds.core.DatasetInfo(
    name='kddcup99',
    full_name='kddcup99/1.0.1',
    description="""
    This is the data set used for The Third International Knowledge Discovery and
    Data Mining Tools Competition, which was held in conjunction with KDD-99 The
    Fifth International Conference on Knowledge Discovery and Data Mining. The
    competition task was to build a network intrusion detector, a predictive model
    capable of distinguishing between 'bad' connections, called intrusions or
    attacks, and 'good' normal connections. This database contains a standard set of
    data to be audited, which includes a wide variety of intrusions simulated in a
    military network environment.
    """,
    homepage='https://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html',
    data_path='C:\\Users\\EhsanQA\\tensorflow_datasets\\kddcup99\\1.0.1',
    file_format=tfrecord,
    download_size=18.62 MiB,
    dataset_size=5.25 GiB,
    features=FeaturesDict({
        'count': int32,
        '

In [13]:
for i, item in ds_train.enumerate():
    if (i > 0):
        break
    print(i)
    print(item)


tf.Tensor(0, shape=(), dtype=int64)
{'count': <tf.Tensor: shape=(), dtype=int32, numpy=2>, 'diff_srv_rate': <tf.Tensor: shape=(), dtype=float32, numpy=0.0>, 'dst_bytes': <tf.Tensor: shape=(), dtype=int32, numpy=0>, 'dst_host_count': <tf.Tensor: shape=(), dtype=int32, numpy=47>, 'dst_host_diff_srv_rate': <tf.Tensor: shape=(), dtype=float32, numpy=0.11>, 'dst_host_rerror_rate': <tf.Tensor: shape=(), dtype=float32, numpy=0.0>, 'dst_host_same_src_port_rate': <tf.Tensor: shape=(), dtype=float32, numpy=0.55>, 'dst_host_same_srv_rate': <tf.Tensor: shape=(), dtype=float32, numpy=0.55>, 'dst_host_serror_rate': <tf.Tensor: shape=(), dtype=float32, numpy=0.0>, 'dst_host_srv_count': <tf.Tensor: shape=(), dtype=int32, numpy=32>, 'dst_host_srv_diff_host_rate': <tf.Tensor: shape=(), dtype=float32, numpy=0.06>, 'dst_host_srv_rerror_rate': <tf.Tensor: shape=(), dtype=float32, numpy=0.0>, 'dst_host_srv_serror_rate': <tf.Tensor: shape=(), dtype=float32, numpy=0.0>, 'duration': <tf.Tensor: shape=(), dtype

In [14]:
auto_encoder1.fit(x_train, x_train, epochs=50, batch_size=128, shuffle=True, validation_data=(x_train, x_train), 
            callbacks=[TensorBoard(log_dir='/tmp/autoencoder')])

NameError: name 'x_train' is not defined